<a href="https://colab.research.google.com/github/krips2398/tensorflow_learnings/blob/master/Deep_neural_network_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#CONVOLUTIONAL NEURAL NETWORKS

It inspects the subsets of images, learns features of image

In [2]:
!pip install tensorflow==1.14.0

In [3]:
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [0]:
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:
#Define a log path to tensorboard log files

In [0]:
logPath ="./tb_logs/"

In [0]:
#   Adds summaries statistics for use in TensorBoard visualization.  
#      From https://www.tensorflow.org/get_started/summaries_and_tensorboard
def variable_summaries(var):
   with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)

In [0]:
sess = tf.InteractiveSession()

In [0]:
#Using Interactive session as default session so that we really do not need to go back and pass the session every single time

Defining the placeholders --> x- represents the 28*28 dimensional vector
y_ is the actual label of the image-->
[0,1,0,0,0,0,0,0,0,0,0] - represents digit 1
Similarly [0,0,1,0,0,0,0,0,0,0,0] - represents digit 2

In [0]:
#Also define the name scopes for different input data that we are using

In [0]:
with tf.name_scope("MNIST_Input"):
  x = tf.placeholder(tf.float32, shape=[None, 784], name="x")
  y_ = tf.placeholder(tf.float32, [None, 10], name="y_")

In [0]:
# change the MNIST input data from a list of values to a 28 pixel X 28 pixel X 1 grayscale value cube
#    which the Convolution NN can use.
with tf.name_scope("Input_Reshape"):
  x_image = tf.reshape(x, [-1,28,28,1], name="x_image")
  tf.summary.image('input_img', x_image, 5)
#here -1 is obviously not a valid dimension --> it is simply used as a flag to reduce it or split the image into 28*28*1

**RELU as Activation Function**


> ReLU stands for rectified linear unit, and is a type of activation function. Mathematically, it is defined as y = max(0, x).


> https://machinelearningmastery.com/rectified-linear-activation-function-for-deep-learning-neural-networks/





**truncated_normal**

> https://docs.w3cub.com/tensorflow~python/tf/truncated_normal/







In [0]:
# Define helper functions to created weights and baises variables, and convolution, and pooling layers
#   We are using RELU as our activation function.  These must be initialized to a small positive number 
#   and with some noise so you don't end up going to zero when comparing diffs
def weight_variable(shape,name=None):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name=name)

def bias_variable(shape,name=None):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=name)

#We are basically picking up the random values for weights and bias variables


**Convolution and Pooling**

> https://stackoverflow.com/questions/34619177/what-does-tf-nn-conv2d-do-in-tensorflow


> Pooling all the conv values to prevent overfitting





In [0]:
#   Convolution and Pooling - we do Convolution, and then pooling to control overfitting
def conv2d(x, W, name=None):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME', name=name)

def max_pool_2x2(x,name=None):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME', name=name)

In [16]:
# Define layers in the NN

# 1st Convolution layer
# 32 features for each 5X5 patch of the image
with tf.name_scope('Conv1'):
  with tf.name_scope('weights'):
    W_conv1 = weight_variable([5, 5, 1, 32])
    variable_summaries(W_conv1)
  with tf.name_scope('baises'):
    b_conv1 = bias_variable([32])
    variable_summaries(b_conv1)
   #Do convolution on images, add bias and push through RELU activation
  conv1_wx_b = conv2d(x_image, W_conv1,name="conv2d") + b_conv1
  tf.summary.histogram('conv1_wx_b', conv1_wx_b)
  h_conv1 = tf.nn.relu(conv1_wx_b, name="relu")
  tf.summary.histogram('h_conv1', h_conv1)
  # take results and run through max_pool
  h_pool1 = max_pool_2x2(h_conv1, name="pool")


# 2nd Convolution layer
with tf.name_scope('Conv2'):
# Process the 32 features from Convolution layer 1, in 5 X 5 patch.  return 64 features weights and biases
    with tf.name_scope('weights'):
        W_conv2 = weight_variable([5, 5, 32, 64], name="weight")
        variable_summaries(W_conv2)
    with tf.name_scope('biases'):
        b_conv2 = bias_variable([64], name="bias")
        variable_summaries(b_conv2)
    # Do convolution of the output of the 1st convolution layer.  Pool results 
    conv2_wx_b = conv2d(h_pool1, W_conv2, name="conv2d") + b_conv2
    tf.summary.histogram('conv2_wx_b', conv2_wx_b)
    h_conv2 = tf.nn.relu(conv2_wx_b, name="relu")
    tf.summary.histogram('h_conv2', h_conv2)
    h_pool2 = max_pool_2x2(h_conv2, name="pool")

with tf.name_scope('FC'):
    # Fully Connected Layer
    W_fc1 = weight_variable([7 * 7 * 64, 1024], name="weight")
    b_fc1 = bias_variable([1024], name="bias")
    #   Connect output of pooling layer 2 as input to full connected layer
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1, name="relu")

# dropout some neurons to reduce overfitting
keep_prob = tf.placeholder(tf.float32, name="keep_prob")  # get dropout probability as a training input.
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

with tf.name_scope("Readout"):
# Readout layer
    W_fc2 = weight_variable([1024, 10], name="weight")
    b_fc2 = bias_variable([10], name="bias")

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
# Define model
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [18]:
# Loss measurement
with tf.name_scope("cross_entropy"):
  cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y_))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [0]:
# loss optimization
with tf.name_scope("loss_optimizer"):
  train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

**Adam Optimization**

> Something better than gradient descent


> https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/





In [0]:
with tf.name_scope("accuracy"):
  # What is correct
  correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
  # How accurate is it?
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [21]:
tf.summary.scalar("cross_entropy_scl", cross_entropy)

<tf.Tensor 'cross_entropy_scl:0' shape=() dtype=string>

In [22]:
tf.summary.scalar("training_accuracy",accuracy)

<tf.Tensor 'training_accuracy:0' shape=() dtype=string>

In [23]:
accuracy

<tf.Tensor 'accuracy/Mean:0' shape=() dtype=float32>

In [24]:
correct_prediction

<tf.Tensor 'accuracy/Equal:0' shape=(?,) dtype=bool>

In [0]:
# tensorboard - merge summaries

In [0]:
summarize_all = tf.summary.merge_all()

In [0]:
# Train the model
import time

#  define number of steps and how often we display progress
num_steps = 3000
display_every = 100


In [0]:
# Start timer
start_time = time.time()
end_time = time.time()

Whwn u get `FailedPreconditionError: Attempting to use uninitialized in Tensorflow` - initialize all the variables declared like below --> run the global initializer
Stack overflow link https://stackoverflow.com/questions/34001922/failedpreconditionerror-attempting-to-use-uninitialized-in-tensorflow

In [0]:
sess.run(tf.global_variables_initializer())

In [0]:
# TB - Write the default graph out so we can view it's structure
tbWriter = tf.summary.FileWriter(logPath, sess.graph)

In [31]:
# Train the model
import time

#  define number of steps and how often we display progress
num_steps = 2000
display_every = 100

# Start timer
start_time = time.time()
end_time = time.time()
for i in range(num_steps):
    batch = mnist.train.next_batch(50)
    _, summary = sess.run([train_step, summarize_all], feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})


    # Periodic status display
    if i%display_every == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})
        end_time = time.time()
        print("step {0}, elapsed time {1:.2f} seconds, training accuracy {2:.3f}%".format(i, end_time-start_time, train_accuracy*100.0))
        # write summary to log
        tbWriter.add_summary(summary,i)

step 0, elapsed time 0.54 seconds, training accuracy 8.000%
step 100, elapsed time 26.92 seconds, training accuracy 88.000%
step 200, elapsed time 53.42 seconds, training accuracy 86.000%
step 300, elapsed time 79.97 seconds, training accuracy 98.000%
step 400, elapsed time 106.35 seconds, training accuracy 90.000%
step 500, elapsed time 132.65 seconds, training accuracy 92.000%
step 600, elapsed time 159.08 seconds, training accuracy 96.000%
step 700, elapsed time 185.48 seconds, training accuracy 98.000%
step 800, elapsed time 211.87 seconds, training accuracy 88.000%
step 900, elapsed time 238.26 seconds, training accuracy 88.000%
step 1000, elapsed time 264.64 seconds, training accuracy 98.000%
step 1100, elapsed time 291.15 seconds, training accuracy 92.000%
step 1200, elapsed time 317.48 seconds, training accuracy 98.000%
step 1300, elapsed time 343.98 seconds, training accuracy 100.000%
step 1400, elapsed time 370.41 seconds, training accuracy 96.000%
step 1500, elapsed time 396

In [32]:
# Display summary 
#     Time to train
end_time = time.time()
print("Total training time for {0} batches: {1:.2f} seconds".format(i+1, end_time-start_time))

#     Accuracy on test data
print("Test accuracy {0:.3f}%".format(accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})*100.0))

sess.close()

Total training time for 2000 batches: 612.07 seconds
Test accuracy 97.740%


In [34]:
!pip install tensorboard==1.14.0

In [35]:
tensorboard.__version__

NameError: ignored

In [36]:
!tensorboard --logdir tb_logs

/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa